In [0]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchtext.datasets import TranslationDataset, IWSLT
from torchtext.data import Field, Iterator
import spacy
import numpy as np
import random
import math
import time
from collections import defaultdict
from nltk.translate.bleu_score import sentence_bleu
import gc

## Seeding
Specify seeds for all random processes to ensure reproducibility.

In [0]:
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

## Load Data
The data is maintained in a google drive at ```My Drive/data/*```. The following sections specifies how to load this data into the appropriate dataset object and fields. Only the europarl section of the WMT14 dataset is used due to compute limitations.

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
# creating torchtext datasets
sos_token='<sos>'
eos_token='<eos>'
pad_token='<pad>'
unk_token='<unk>'

def tokenize(text):
    return text.split()

# The paper originally maintains case however this increases the size of the 
# vocabulary and hence we chose against this method.
sourceLanguage = targetLanguage = Field(sequential=True, 
                                        use_vocab=True, 
                                        init_token=sos_token, 
                                        eos_token=eos_token, 
                                        fix_length=None, 
                                        dtype=torch.long, 
                                        lower=True, 
                                        tokenize=tokenize,
                                        pad_token=pad_token, 
                                        unk_token=unk_token)

# Filters out sentences longer than 50 in the training data.
dataset = TranslationDataset("/content/gdrive/My Drive/data/europarl-v7.fr-en", 
                             exts=('.en', '.fr'), 
                             fields=(sourceLanguage, targetLanguage),
                             filter_pred=lambda x: len(x.__dict__['src']) <= 50)

In [0]:
sourceLanguage.build_vocab(dataset, min_freq = 2, max_size = 30000)    
targetLanguage.build_vocab(dataset, min_freq = 2, max_size = 30000)

In [0]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 80
INPUT_DIM  = len(sourceLanguage.vocab)
OUTPUT_DIM = len(targetLanguage.vocab) 
EMB_DIM = 256
HID_DIM = 512
MAXOUT_DIM = 400
ATT_HID_DIM = 1000
MAX_LENGTH = 50

In [0]:
train_iterator = Iterator(
    dataset, 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.src),
    device = device
)

## ih

In [0]:
class EncoderRNNEncDec(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim):
        super(EncoderRNNEncDec, self).__init__()
        self.embedding  = nn.Embedding(input_dim, emb_dim)
        self.rnn        = nn.GRU(emb_dim, hid_dim)
        self.fc_out     = nn.Linear(hid_dim, hid_dim)
        self.activation = nn.Tanh()
        
    def forward(self, input):
        output          = self.embedding(input)
        output, hidden  = self.rnn(output)
        output          = self.activation(self.fc_out(hidden.squeeze(0)))
        output          = output.unsqueeze(0)
        return output

class DecoderRNNEncDec(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, max_dim):
        super(DecoderRNNEncDec, self).__init__()
        self.embedding  = nn.Embedding(output_dim, emb_dim)
        self.rnn        = nn.GRU(emb_dim + hid_dim, hid_dim)
        self.max_dim    = max_dim
        self.max_out    = nn.Linear(emb_dim + hid_dim * 2, 2 * max_dim)
        self.out        = nn.Linear(max_dim, output_dim)

    def forward(self, input, hidden, context):
        embedded        = self.embedding(input)                     
        output          = torch.cat((embedded, context), dim = 2)
        _, hidden       = self.rnn(output, hidden)
        output          = torch.cat((embedded.squeeze(0), 
                                     hidden.squeeze(0), 
                                     context.squeeze(0)), 
                                    dim = 1)
        output          = self.max_out(output)
        output          = output.view(input.shape[1], self.max_dim, 2)
        output, _       = torch.max(output, 2)
        output          = self.out(output)
        return output, hidden

class Seq2Seq(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, output_dim, max_dim):
        super(Seq2Seq, self).__init__()
        self.fc_in      = nn.Linear(hid_dim, hid_dim)
        self.fc_act     = nn.Tanh()
        self.encoder    = EncoderRNNEncDec(input_dim, emb_dim, hid_dim)
        self.decoder    = DecoderRNNEncDec(output_dim, emb_dim, hid_dim, max_dim)
        self.output_dim = output_dim
    
    def forward(self, src, trg, is_train=False):
        context         = self.encoder(src)
        decoder_hidden  = self.fc_in(context.squeeze(0))
        decoder_hidden  = self.fc_act(decoder_hidden).unsqueeze(0)
        outputs         = torch.zeros(trg.shape[0], 
                                      trg.shape[1], 
                                      self.output_dim).to(device)
        input           = trg[0]
        for t in range(1, trg.shape[0]):
            decoder_output, decoder_hidden = self.decoder(input.unsqueeze(0), 
                                                          decoder_hidden, 
                                                          context)
            outputs[t] = decoder_output
            input = trg[t] if is_train else decoder_output.argmax(1)
        return outputs

In [0]:
model       = Seq2Seq(INPUT_DIM, EMB_DIM, HID_DIM, OUTPUT_DIM, MAXOUT_DIM).to(device)
optimizer   = optim.Adadelta(model.parameters(), rho=0.95, eps=1e-06)
TRG_PAD_IDX = targetLanguage.vocab.stoi[targetLanguage.pad_token]
criterion   = nn.CrossEntropyLoss(ignore_index = TRG_PAD_IDX)

In [0]:
def train(model, iterator, optimizer, criterion):
    model.train()
    epoch_loss = 0.0
    for i, batch in enumerate(iterator):
        optimizer.zero_grad()
        src, trg    = batch.src, batch.trg
        outputs     = model(src, trg, is_train=True)
        output_dim  = outputs.shape[-1]
        outputs     = outputs[1:].view(-1, output_dim) # Reshape to match trg
        trg         = trg[1:].view(-1) # Ignore the init token.
        loss        = criterion(outputs, trg)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    return epoch_loss / len(iterator)

In [0]:
# Convenience to get runtime for training.
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [0]:
model.load_state_dict(torch.load('/content/gdrive/My Drive/data/control-model-monodirectional-0.pt'))
model.eval()

Seq2Seq(
  (fc_in): Linear(in_features=512, out_features=512, bias=True)
  (fc_act): Tanh()
  (encoder): EncoderRNNEncDec(
    (embedding): Embedding(30004, 256)
    (rnn): GRU(256, 512)
    (fc_out): Linear(in_features=512, out_features=512, bias=True)
    (activation): Tanh()
  )
  (decoder): DecoderRNNEncDec(
    (embedding): Embedding(30004, 256)
    (rnn): GRU(768, 512)
    (max_out): Linear(in_features=1280, out_features=800, bias=True)
    (out): Linear(in_features=400, out_features=30004, bias=True)
  )
)

In [0]:
N_EPOCHS = 10

for epoch in range(1, N_EPOCHS):
    train_iterator.init_epoch() # Processes like shuffling that happen before epoch.
    start_time = time.time()
    train_loss = train(model, train_iterator, optimizer, criterion) 
    end_time = time.time()
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss: }')

    # Checkpoint to ensure progrerss isn't lost.
    torch.save(model.state_dict(), f'/content/gdrive/My Drive/data/control-model-monodirectional-{epoch}.pt')
    model.load_state_dict(torch.load(f'/content/gdrive/My Drive/data/control-model-monodirectional-{epoch}.pt'))
    model.eval()

Epoch: 02 | Time: 197m 1s
	Train Loss:  3.0190634663087166
Epoch: 03 | Time: 196m 30s
	Train Loss:  2.770725098545351
Epoch: 04 | Time: 195m 0s
	Train Loss:  2.6412053691687816
Epoch: 05 | Time: 194m 12s
	Train Loss:  2.556920369685852
